In [3]:
from pykrx import stock
from pykrx import bond
import pandas as pd
import math
import datetime as dt
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, GRU



from itertools import cycle

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from matplotlib import pyplot as plt
import seaborn as sns

# LG화학, KB금융 feature_selection 5가지를 이용한 변수중요도 추출

In [30]:
df = pd.read_csv('./stock_v08.csv', encoding='cp949')
df_lg = df[df['종목']=='LG화학']
df_kb = df[df['종목']=='KB금융']
df_lg

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률,BPS,PER,...,SMA,WMA,MOM,STCK,STCD,SIG,LWR,ADO,RSI,CCI
1209,2018-05-08,332000,336000,330500,330500,356424,1.190000e+11,-3.08,206544,13.03,...,352100.0,345754.5455,-42000,11.929825,13.897714,357344.3490,-100.000000,-371464.480900,27.465013,-173.193341
1210,2018-05-09,328500,341500,326500,339500,242892,8.202752e+10,2.72,206544,13.38,...,348800.0,343463.6364,-33000,13.214228,12.488678,353049.2532,-76.146789,-313221.109100,34.697680,-131.857513
1211,2018-05-10,339500,344500,337500,340000,189496,6.464441e+10,0.15,206544,13.40,...,345350.0,341863.6364,-34500,16.207951,13.784001,348270.3876,-75.229358,-278146.538000,35.084946,-94.406034
1212,2018-05-11,343500,347500,341500,345000,214165,7.384304e+10,1.47,206544,13.60,...,344600.0,341800.0000,-7500,28.066925,19.163035,326500.0000,-64.423077,-227154.566000,38.981883,-67.471679
1213,2018-05-14,349000,357500,347000,354500,302609,1.070000e+11,2.75,206544,13.97,...,344700.0,343600.0000,1000,38.782522,27.685799,326920.0000,-44.000000,-144378.935200,45.657014,-21.520929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2413,2023-03-27,702000,712000,691000,698000,175698,1.230000e+11,1.16,278455,14.82,...,702100.0,698709.0909,-20000,33.796296,32.543271,743054.1043,-65.277778,-70909.336450,50.845567,-10.003651
2414,2023-03-28,701000,704000,687000,704000,238741,1.660000e+11,0.86,278455,14.94,...,702000.0,699054.5455,-1000,32.671958,33.531746,740251.9401,-56.944444,3596.015522,52.375400,-31.818182
2415,2023-03-29,704000,708000,695000,705000,146056,1.030000e+11,0.14,278455,14.97,...,700400.0,699600.0000,-16000,40.740741,35.736332,737561.8625,-55.555556,58772.147610,52.639962,-19.133795
2416,2023-03-30,708000,709000,692000,696000,181406,1.270000e+11,-1.28,278455,14.77,...,697000.0,698800.0000,-34000,39.814815,37.742504,734979.3880,-68.055556,45443.583950,49.950517,-49.971782


In [31]:
df_kb

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률,BPS,PER,...,SMA,WMA,MOM,STCK,STCD,SIG,LWR,ADO,RSI,CCI
3627,2018-05-08,58300,59000,58100,58400,1428139,8.339094e+10,-1.35,81411,7.03,...,60240.0,59929.09091,-2100,33.249300,54.387943,61540.00000,-91.428571,-4.788429e+05,42.013449,-87.627583
3628,2018-05-09,57900,58500,57100,58200,1254153,7.248766e+10,-0.34,81411,7.01,...,59980.0,59558.18182,-2600,20.809213,36.320158,61402.40000,-75.555556,-2.514213e+05,41.109107,-142.459212
3629,2018-05-10,58100,59300,58000,58000,945925,5.520594e+10,-0.34,81411,6.98,...,59650.0,59198.18182,-3300,17.671958,23.910157,61144.25600,-80.000000,-4.365048e+05,40.177753,-108.591065
3630,2018-05-11,58000,58500,57800,58200,544837,3.170498e+10,0.34,81411,7.01,...,59420.0,58934.54545,-2300,22.962963,20.481378,60901.60064,-75.555556,-4.477732e+05,41.602557,-128.017358
3631,2018-05-14,58000,58700,58000,58200,593203,3.456684e+10,0.00,81411,7.01,...,59230.0,58712.72727,-1900,22.962963,21.199295,60673.50460,-75.555556,-4.925677e+05,41.602557,-107.040092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4831,2023-03-27,46650,47000,46350,46700,1477169,6.907272e+10,-0.74,121808,4.19,...,48260.0,48057.27273,-3600,12.996103,21.190083,49486.38735,-93.203884,-1.881988e+06,32.278804,-180.598221
4832,2023-03-28,47500,48450,47450,47900,1680191,8.057562e+10,2.57,121808,4.30,...,48210.0,47991.81818,-500,12.668105,16.185700,49047.29312,-69.902913,-1.753691e+06,40.305034,-90.416299
4833,2023-03-29,47600,48200,47600,48050,1092869,5.241417e+10,0.31,121808,4.32,...,48115.0,47962.72727,-950,25.631068,17.098425,48669.67208,-60.000000,-1.367914e+06,41.242478,-82.801120
4834,2023-03-30,47800,47900,47050,47200,1629175,7.710546e+10,-1.77,121808,4.24,...,48030.0,47796.36364,-850,30.032362,22.777178,48450.00000,-80.000000,-1.421159e+06,37.635697,-110.684308


In [32]:
aa = df_lg.loc[~(df_lg["날짜"].isin(df_kb['날짜']))]
aa

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률,BPS,PER,...,SMA,WMA,MOM,STCK,STCD,SIG,LWR,ADO,RSI,CCI


LG화학, KB금융 모두 같은날짜 포함인걸 확인

In [33]:
df_lg.drop(['날짜','업종','종목', 'MA10'], axis=1, inplace=True)
df_kb.drop(['날짜','업종','종목', 'MA10'], axis=1, inplace=True)
df_lg

C:\Users\magne\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,시가,고가,저가,종가,거래량,거래대금,등락률,BPS,PER,PBR,...,SMA,WMA,MOM,STCK,STCD,SIG,LWR,ADO,RSI,CCI
1209,332000,336000,330500,330500,356424,1.190000e+11,-3.08,206544,13.03,1.60,...,352100.0,345754.5455,-42000,11.929825,13.897714,357344.3490,-100.000000,-371464.480900,27.465013,-173.193341
1210,328500,341500,326500,339500,242892,8.202752e+10,2.72,206544,13.38,1.64,...,348800.0,343463.6364,-33000,13.214228,12.488678,353049.2532,-76.146789,-313221.109100,34.697680,-131.857513
1211,339500,344500,337500,340000,189496,6.464441e+10,0.15,206544,13.40,1.65,...,345350.0,341863.6364,-34500,16.207951,13.784001,348270.3876,-75.229358,-278146.538000,35.084946,-94.406034
1212,343500,347500,341500,345000,214165,7.384304e+10,1.47,206544,13.60,1.67,...,344600.0,341800.0000,-7500,28.066925,19.163035,326500.0000,-64.423077,-227154.566000,38.981883,-67.471679
1213,349000,357500,347000,354500,302609,1.070000e+11,2.75,206544,13.97,1.72,...,344700.0,343600.0000,1000,38.782522,27.685799,326920.0000,-44.000000,-144378.935200,45.657014,-21.520929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2413,702000,712000,691000,698000,175698,1.230000e+11,1.16,278455,14.82,2.51,...,702100.0,698709.0909,-20000,33.796296,32.543271,743054.1043,-65.277778,-70909.336450,50.845567,-10.003651
2414,701000,704000,687000,704000,238741,1.660000e+11,0.86,278455,14.94,2.53,...,702000.0,699054.5455,-1000,32.671958,33.531746,740251.9401,-56.944444,3596.015522,52.375400,-31.818182
2415,704000,708000,695000,705000,146056,1.030000e+11,0.14,278455,14.97,2.53,...,700400.0,699600.0000,-16000,40.740741,35.736332,737561.8625,-55.555556,58772.147610,52.639962,-19.133795
2416,708000,709000,692000,696000,181406,1.270000e+11,-1.28,278455,14.77,2.50,...,697000.0,698800.0000,-34000,39.814815,37.742504,734979.3880,-68.055556,45443.583950,49.950517,-49.971782


해당 업종들도 마찬가지로 날짜, 업종, 종목을 제외한 나머지 변수를 변수선택법 진행

# 1. LG화학 변수선택

In [34]:
from sklearn.model_selection import train_test_split

X = df_lg
y = df_lg['종가']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
cols = X_train.columns

In [35]:
from sklearn.ensemble import RandomForestRegressor


rf = RandomForestRegressor(n_estimators=100, random_state=0)

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

In [36]:
importance = rf.feature_importances_
feature = X_train.columns
importances = pd.DataFrame()
importances['feature'] = feature
importances['importances'] = importance
importances.sort_values('importances', ascending=False, inplace=True)
importances.reset_index(drop=True, inplace=True)
importances

,feature,importances
0,종가,9.180806e-01
1,고가,4.928782e-02
2,저가,1.626284e-02
3,시가,9.012095e-03
4,MA5,2.361210e-03
5,DIV,1.951413e-03
6,SMA,1.059538e-03
7,PBR,6.749810e-04
8,WMA,6.286451e-04
9,PER,5.455848e-04


In [37]:
from sklearn.feature_selection import RFE, RFECV

min_features_to_select = 10
step = 1
selector = RFECV(rf, step=step, cv=5, min_features_to_select=min_features_to_select)
selector = selector.fit(X_train, y_train)
selector.support_

array([ True,  True,  True,  True, False, False, False, False,  True,
        True, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False,  True,  True, False, False,
       False, False,  True, False, False, False])

In [38]:
a = selector.support_.reshape(1,-1)
df_a = pd.DataFrame(columns=X_train.columns, data=a)
df_a
true_columns = df_a.columns[df_a.iloc[0]].tolist()

print(true_columns)

['시가', '고가', '저가', '종가', 'PER', 'PBR', 'DIV', 'MA5', 'SMA', 'WMA', 'LWR']


In [39]:
from sklearn.feature_selection import RFE
from sklearn.svm import LinearSVR
# X_train = np.array(X_train)
# y_train = np.array(y_train)
estimator = LinearSVR()
selector = RFE(estimator, n_features_to_select=10, step=1)
selector = selector.fit(X_train, y_train)
selector.support_

C:\Users\magne\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\magne\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\magne\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\magne\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\magne\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False,  True,  True,  True,  True,  True, False,
       False,  True,  True,  True, False,  True])

In [40]:
a = selector.support_.reshape(1,-1)
df_a = pd.DataFrame(columns=X_train.columns, data=a)
df_a
true_columns = df_a.columns[df_a.iloc[0]].tolist()

print(true_columns)

['short_거래량', 'MA5', 'MA20', 'SMA', 'WMA', 'MOM', 'SIG', 'LWR', 'ADO', 'CCI']


In [41]:
from sklearn.linear_model import Ridge, Lasso


ridge_alpha = 0.01
lasso_alpha = 0.001

rid = Ridge(alpha = ridge_alpha)
las = Lasso(alpha = lasso_alpha)

scaler=MinMaxScaler(feature_range=(0,1))
X_train=scaler.fit_transform(np.array(X_train))
y_train=scaler.fit_transform(np.array(y_train).reshape(-1,1))

rid.fit(X_train,y_train)
las.fit(X_train,y_train)

X_train= scaler.inverse_transform(X_train)
y_train= scaler.inverse_transform(y_train)

rid.coef_, las.coef_

(array([[-1.10060141e-02,  1.09435993e-01,  8.65590433e-02,
          6.08726322e-01, -1.24201921e-03,  1.30912581e-03,
          1.58406149e-02,  3.51023594e-02, -2.37324106e-02,
          1.79775877e-01, -5.49083742e-03, -8.14437342e-03,
          1.05141132e-03, -1.79899689e-04, -7.92613293e-04,
         -3.28645543e-03,  3.81842849e-03, -3.38280531e-03,
          4.02017576e-03, -1.51690412e-03,  1.29658097e-03,
          1.30912581e-03,  0.00000000e+00,  3.61222363e-03,
         -2.07691069e-03,  0.00000000e+00, -2.07691069e-03,
         -1.54798519e-03, -1.24201921e-03,  7.41208809e-04,
          2.20025373e-02,  1.06474929e-02, -2.10450414e-03,
          1.88874640e-02,  6.77030068e-03,  3.45675120e-03,
         -4.77358520e-03, -4.98668370e-04,  5.26825980e-04,
          4.79942203e-03, -5.65852351e-04, -4.35202395e-04]]),
 array([ 0.        ,  0.        ,  0.        ,  0.96435147,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        -0.

In [42]:
a = rid.coef_.reshape(1,-1)
df_a = pd.DataFrame(columns=cols, data=a)
df_a

,시가,고가,저가,종가,거래량,거래대금,등락률,BPS,PER,PBR,...,SMA,WMA,MOM,STCK,STCD,SIG,LWR,ADO,RSI,CCI
0,-0.011006,0.109436,0.086559,0.608726,-0.001242,0.001309,0.015841,0.035102,-0.023732,0.179776,...,-0.002105,0.018887,0.00677,0.003457,-0.004774,-0.000499,0.000527,0.004799,-0.000566,-0.000435


In [43]:
aaa = df_a.columns.tolist()

fli=[]
coli=[]
for i, col in enumerate(aaa) :
    fli.append(col)
    coli.append(df_a.iloc[0,i])
    
dd = pd.DataFrame({'feature':fli, 'coef':coli})

dd.sort_values('coef', ascending=False, inplace=True)
dd.reset_index(drop=True, inplace=True)
dd


,feature,coef
0,종가,0.608726
1,PBR,0.179776
2,고가,0.109436
3,저가,0.086559
4,BPS,0.035102
5,MA5,0.022003
6,WMA,0.018887
7,등락률,0.015841
8,MA20,0.010647
9,MOM,0.006770


In [44]:

b = las.coef_.reshape(1,-1)
df_b = pd.DataFrame(columns=cols, data=b)
df_b
true_columns = df_b.columns[df_b.iloc[0] != 0].tolist()
print(true_columns)

['종가', 'MA20', 'SMA', 'WMA']


RF feature_importances_ 기준 상위 10개 feature

종가, 고가, 저가, 시가, MA5, DIV, SMA, PBR, WMA, PER

RF RFECV 기준 11개 feature

시가, 고가, 저가, 종가, PER, PBR, DIV, MA5, SMA, WMA, LWR

LinearSVR RFE 기준 10개 feature

short_거래량, MA5, MA20, SMA, WMA, MOM, SIG, LWR, ADO, CCI

Lasso coef 기준 4개 feature

종가, WMA, SMA, MA20

Ridge coef 기준 상위 5개 feature

종가, 고가, 저가, 시가, PER, PBR, BPS, MA5, WMA, 등락률

------------------------------------------------------------------------

5개 포함 : WMA

4개 포함 : 종가, MA5, SMA

3개 포함 : 고가, 저가, 시가, PER, PBR

2개 포함 : DIV, LWR, MA20

1개 포함 : short_거래량, MOM, SIG, ADO, CCI, BPS, 등락률

# 가장 포함이 많이 된 상위 9개 변수 추출 완료

종가, 고가, 저가, 시가, WMA, MA5, SMA, PER, PBR

# 2. KB금융 변수선택

In [45]:
from sklearn.model_selection import train_test_split

X = df_kb
y = df_kb['종가']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
cols = X_train.columns

In [46]:
from sklearn.ensemble import RandomForestRegressor


rf = RandomForestRegressor(n_estimators=100, random_state=0)

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

In [47]:
importance = rf.feature_importances_
feature = X_train.columns
importances = pd.DataFrame()
importances['feature'] = feature
importances['importances'] = importance
importances.sort_values('importances', ascending=False, inplace=True)
importances.reset_index(drop=True, inplace=True)
importances

,feature,importances
0,종가,9.202567e-01
1,고가,5.092330e-02
2,저가,1.967265e-02
3,시가,3.841995e-03
4,MA5,1.979199e-03
5,DIV,1.483542e-03
6,PER,7.970711e-04
7,PBR,4.137994e-04
8,foreign_보유수량,1.339364e-04
9,MOM,9.051614e-05


In [48]:
from sklearn.feature_selection import RFE, RFECV

min_features_to_select = 10
step = 1
selector = RFECV(rf, step=step, cv=5, min_features_to_select=min_features_to_select)
selector = selector.fit(X_train, y_train)
selector.support_

array([ True,  True,  True,  True, False, False, False, False,  True,
        True, False,  True, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False,  True, False, False, False,  True, False,
       False, False, False, False, False, False])

In [49]:
a = selector.support_.reshape(1,-1)
df_a = pd.DataFrame(columns=X_train.columns, data=a)
df_a
true_columns = df_a.columns[df_a.iloc[0]].tolist()

print(true_columns)

['시가', '고가', '저가', '종가', 'PER', 'PBR', 'DIV', 'foreign_보유수량', 'MA5', 'MOM']


In [50]:
from sklearn.feature_selection import RFE
from sklearn.svm import LinearSVR
# X_train = np.array(X_train)
# y_train = np.array(y_train)
estimator = LinearSVR()
selector = RFE(estimator, n_features_to_select=10, step=1)
selector = selector.fit(X_train, y_train)
selector.support_

C:\Users\magne\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\magne\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\magne\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\magne\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\magne\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True,  True])

In [51]:
a = selector.support_.reshape(1,-1)
df_a = pd.DataFrame(columns=X_train.columns, data=a)
df_a
true_columns = df_a.columns[df_a.iloc[0]].tolist()

print(true_columns)

['MA20', 'SMA', 'WMA', 'MOM', 'STCK', 'STCD', 'SIG', 'LWR', 'RSI', 'CCI']


In [52]:
from sklearn.linear_model import Ridge, Lasso


ridge_alpha = 0.01
lasso_alpha = 0.001

rid = Ridge(alpha = ridge_alpha)
las = Lasso(alpha = lasso_alpha)

scaler=MinMaxScaler(feature_range=(0,1))
X_train=scaler.fit_transform(np.array(X_train))
y_train=scaler.fit_transform(np.array(y_train).reshape(-1,1))

rid.fit(X_train,y_train)
las.fit(X_train,y_train)

X_train= scaler.inverse_transform(X_train)
y_train= scaler.inverse_transform(y_train)

rid.coef_, las.coef_

(array([[-1.82906412e-02,  8.23068245e-02,  9.38867646e-02,
          7.02318405e-01,  2.18107708e-03, -1.94549278e-03,
          2.30830998e-02,  2.19176522e-04,  4.01882196e-02,
          1.22716882e-03,  2.21604087e-02, -2.95789935e-03,
         -1.15692637e-03, -8.90291231e-04, -1.30458611e-03,
          5.11943822e-04,  9.56915443e-04, -2.31730135e-03,
         -3.00959288e-03,  2.39683841e-03, -9.95715683e-04,
         -1.94549278e-03, -1.84203643e-03,  1.66677431e-02,
         -9.94251146e-03, -1.84203643e-03, -9.94251146e-03,
          4.94410822e-03,  2.18107708e-03, -3.39294536e-03,
          4.58213629e-02,  1.38047802e-02, -1.22306745e-02,
          3.82595185e-02,  3.94734066e-03,  7.98739594e-03,
         -1.01979333e-02, -2.52273582e-03,  1.34117925e-03,
          5.12396599e-03,  6.96964289e-03,  2.01725209e-04]]),
 array([ 0.        ,  0.00696178,  0.        ,  0.91305695, -0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.

In [53]:
a = rid.coef_.reshape(1,-1)
df_a = pd.DataFrame(columns=cols, data=a)
df_a

,시가,고가,저가,종가,거래량,거래대금,등락률,BPS,PER,PBR,...,SMA,WMA,MOM,STCK,STCD,SIG,LWR,ADO,RSI,CCI
0,-0.018291,0.082307,0.093887,0.702318,0.002181,-0.001945,0.023083,0.000219,0.040188,0.001227,...,-0.012231,0.03826,0.003947,0.007987,-0.010198,-0.002523,0.001341,0.005124,0.00697,0.000202


In [54]:
aaa = df_a.columns.tolist()

fli=[]
coli=[]
for i, col in enumerate(aaa) :
    fli.append(col)
    coli.append(df_a.iloc[0,i])
    
dd = pd.DataFrame({'feature':fli, 'coef':coli})

dd.sort_values('coef', ascending=False, inplace=True)
dd.reset_index(drop=True, inplace=True)
dd


,feature,coef
0,종가,0.702318
1,저가,0.093887
2,고가,0.082307
3,MA5,0.045821
4,PER,0.040188
5,WMA,0.038260
6,등락률,0.023083
7,EPS,0.022160
8,foreign_보유수량,0.016668
9,MA20,0.013805


In [55]:

b = las.coef_.reshape(1,-1)
df_b = pd.DataFrame(columns=cols, data=b)
df_b
true_columns = df_b.columns[df_b.iloc[0] != 0].tolist()
print(true_columns)

['고가', '종가', 'MA20', 'WMA']


RF feature_importances_ 기준 상위 10개 feature

종가, 고가, 저가, 시가, MA5, DIV, PER, PBR, foreign_보유수량, MOM

RF RFECV 기준 10개 feature

종가, 시가, 저가, 고가, MA5, MOM, PER, PBR, DIV, foreign_보유수량

LinearSVR RFE 기준 10개 feature

MA20, SMA, WMA, MOM, STCK, STCD, SIG, LWR, RSI, CCI

Lasso coef 기준 3개 feature

종가, 고가, MA20, WMA

Ridge coef 기준 상위 5개 feature

종가, 저가, 고가, 시가, MA5, PER, WMA, 등락률, EPS, foreign_보유수량

------------------------------------------------------------------------

4개 포함 : 종가, 고가

3개 포함 : 저가, 시가, MA5, PER, foreign_보유수량, MOM, WMA

2개 포함 : DIV, PBR, MA20

1개 포함 : SMA, STCK, STCD, SIG, LWR, RSI, CCI, 등락률, EPS

# 종가, 고가, 저가, 시가, MA5, WMA, MOM, PER, foreign_보유수량